# Auto iBLR Gaussian Implementation Test 

In [1]:
using Random,LinearAlgebra, Flux.Optimise, Plots, ForneyLab
using ForneyLab:ParamStr2,renderCVI,naturalToBCParams,oneWindowSimulation_MCMC,Pareto_k_fit
import ForneyLab:bcToNaturalParams,naturalParams,bcToStandardDist,sampleBCDist 

In [2]:
Random.seed!(1234);

# Multivariate Results

In [3]:
d=2
if d>=2
    μ_in = ones(d) #[0.0,0.0]
    s_in = 100*diageye(d)  #100*[1 0;0 1.]
    λ_init = [s_in*μ_in;vec(-0.5*s_in)] # Natparams
    msg_in = Message(Multivariate, GaussianWeightedMeanPrecision,xi=s_in*μ_in,w=s_in)
else
    μ_in = 0.0
    s_in = 0.01
    λ_init = [s_in*μ_in,-0.5*s_in] # Natparams
    msg_in = Message(Univariate, GaussianWeightedMeanPrecision,xi=s_in*μ_in,w=s_in)
end



Message: 𝒩(xi=[1.00e+02, 1.00e+02], w=diag[1.00e+02, 1.00e+02])


In [4]:
f(x)= norm(x)
if d>=2
    μ_out = [f(120.)+randn() for i in 1:d]#[f(120.) .+ randn(),f(120.) .+ randn()]
    s_out = 1e12*diageye(d) #[1 0;0 1.]
    msg_out = Message(Multivariate, GaussianWeightedMeanPrecision,xi=s_out*μ_out,w=s_out)
else
    μ_out = f(120.) .+ randn()
    s_out = 100
    msg_out = Message(Univariate, GaussianWeightedMeanPrecision,xi=s_out*μ_out,w=s_out) 
end

logp_nc(z) = logPdf(msg_out.dist, f.(z))
println(msg_out.dist)

𝒩(xi=[1.21e+14, 1.19e+14], w=diag[1.00e+12, 1.00e+12])



### 1e-7 Step Size

In [5]:
max_iterations = 1e5
for str in ["free_energy","none","MonteCarlo"]
opt = ParamStr2(eta=1e-7,pareto_k_thr=0.05,max_iterations=max_iterations,mcmc_window_len=2000,
    rhat_cutoff=1.5,mcse_cutoff =1e-3,ess_threshold=150.,convergence_algo=str,verbose=true)
λ = renderCVI(logp_nc,Int64(max_iterations),opt,λ_init,msg_in)
λ_bc = naturalToBCParams(msg_in.dist,λ)
println("μ=$(λ_bc[1]),S=$(λ_bc[2])")
end
    


FE (1.086811452011988e14) is smaller now,FE_check starts when i=5200
Importance ratios are 0, fitted Pareto shape parameter = NaN
μ=[120.27455910196221, 118.51421725861942],S=[9.99993150993619e9 0.0; 0.0 9.99993150993619e9]
Importance ratios are 0, fitted Pareto shape parameter = NaN
μ=[120.2745589142024, 118.51421538529854],S=[9.99993150993619e9 0.0; 0.0 9.99993150993619e9]
             VI result might be inaccurate!
μ=[120.27157754942513, 118.51127953055705],S=[9.950479110851395e9 0.0; 0.0 9.950479110851395e9]


### 0.5 Step Size

In [6]:
max_iterations = 1e5
for str in ["free_energy","none","MonteCarlo"]
opt = ParamStr2(eta=0.5,pareto_k_thr=0.05,max_iterations=max_iterations,mcmc_window_len=2000,
    rhat_cutoff=1.5,mcse_cutoff =1e-3,ess_threshold=150.,convergence_algo=str,verbose=true)
#opt= Descent(1e-7)
λ = renderCVI(logp_nc,Int64(max_iterations),opt,λ_init,msg_in)
λ_bc = naturalToBCParams(msg_in.dist,λ)
println("μ=$(λ_bc[1]),S=$(λ_bc[2])")
end

FE (1.4157655916345334e6) is smaller now,FE_check starts when i=5200
Algorithm converged at iteration 9200
Importance ratios are 0, fitted Pareto shape parameter = NaN
μ=[120.86734686783716, 119.09825595213465],S=[1.0000000001e12 0.0; 0.0 1.0000000001e12]
Importance ratios are 0, fitted Pareto shape parameter = NaN
μ=[120.86734725221505, 119.09825700563239],S=[1.0000000001e12 0.0; 0.0 1.0000000001e12]
VI converged.
μ=[120.86734718315336, 119.09825616764302],S=[1.0000000001e12 0.0; 0.0 1.0000000001e12]


# Univariate Results

In [7]:
d=1
if d==2
    μ_in = [0.0,0.0]
    s_in = 100*[1 0;0 1.]
    λ_init = [s_in*μ_in;vec(-0.5*s_in)] # Natparams
    msg_in = Message(Multivariate, GaussianWeightedMeanPrecision,xi=s_in*μ_in,w=s_in)
else
    μ_in = 0.0
    s_in = 0.01
    λ_init = [s_in*μ_in,-0.5*s_in] # Natparams
    msg_in = Message(Univariate, GaussianWeightedMeanPrecision,xi=s_in*μ_in,w=s_in)
end
#f(x) = x
# σ(x) = 1/(1+exp(-x))
# f(x)= [x^2+σ(x)tanh(x)+2*log(abs(x))*x,x]
#f(x)= (x^3)*exp(-0.005*abs(x))*-1
f(x)=norm(x)
if d==2
    μ_out = [f(120.) .+ randn(),f(120.) .+ randn()]
    s_out = 1e12*[1 0;0 1.]
    msg_out = Message(Multivariate, GaussianWeightedMeanPrecision,xi=s_out*μ_out,w=s_out)
else
    μ_out = f(120.) .+ randn()
    s_out = 100
    msg_out = Message(Univariate, GaussianWeightedMeanPrecision,xi=s_out*μ_out,w=s_out) 
end


logp_nc(z) = logPdf(msg_out.dist, f.(z))
println(msg_out.dist)


𝒩(xi=1.19e+04, w=100)



### 1e-7 Step Size

In [8]:
max_iterations = 1e3
for str in ["free_energy","none","MonteCarlo"]
opt = ParamStr2(eta=1e-7,pareto_k_thr=0.05,max_iterations=max_iterations,mcmc_window_len=2000,
    rhat_cutoff=1.5,mcse_cutoff =1e-3,ess_threshold=150.,convergence_algo=str,verbose=true)
#opt= Descent(1e-7)
λ = renderCVI(logp_nc,Int64(max_iterations),opt,λ_init,msg_in)
λ_bc = naturalToBCParams(msg_in.dist,λ)
println("μ=$(λ_bc[1]),S=$(λ_bc[2])")
end
    


FE (583472.0273407327) is smaller now,FE_check starts when i=58
Importance ratios are 0, fitted Pareto shape parameter = NaN
μ=-31.30725128826589,S=0.02000296664809481
Importance ratios are 0, fitted Pareto shape parameter = NaN
μ=31.77012756913559,S=0.02000296664809481
             VI result might be inaccurate!
μ=0.8129151431289064,S=0.02500838210516391


### 0.5 Step Size

In [9]:
max_iterations = 1e3
for str in ["free_energy","none","MonteCarlo"]
opt = ParamStr2(eta=0.5,pareto_k_thr=0.05,max_iterations=max_iterations,mcmc_window_len=2000,
    rhat_cutoff=1.5,mcse_cutoff =1e-3,ess_threshold=150.,convergence_algo=str,verbose=true)
#opt= Descent(1e-7)
λ = renderCVI(logp_nc,Int64(max_iterations),opt,λ_init,msg_in)
λ_bc = naturalToBCParams(msg_in.dist,λ)
println("μ=$(λ_bc[1]),S=$(λ_bc[2])")
end

FE (713111.3251198871) is smaller now,FE_check starts when i=52
Algorithm converged at iteration 92
Warning, fitted Pareto shape parameter =0.18546962295530448>=0.05!
μ=118.84569607416516,S=100.01000000000002
Warning, fitted Pareto shape parameter =0.34068557996939935>=0.05!
μ=118.92292353499404,S=100.01000000000002
             VI result might be inaccurate!
μ=23.77809518644229,S=100.01000000000023


## With Initial Step Size found by InexactLineSearch

In [10]:
max_iterations = 1e5
for str in ["free_energy","none","MonteCarlo"]
opt = ParamStr2(pareto_k_thr=0.05,max_iterations=max_iterations,mcmc_window_len=2000,
    rhat_cutoff=1.5,mcse_cutoff =1e-3,ess_threshold=150.,convergence_algo=str,verbose=true)
#opt= Descent(1e-7)
λ = renderCVI(logp_nc,Int64(max_iterations),opt,λ_init,msg_in)
λ_bc = naturalToBCParams(msg_in.dist,λ)
println("μ=$(λ_bc[1]),S=$(λ_bc[2])")
end

inexactLineSearch succeeded, setting initial stepsize to 1.0280139715407661e-5
Burninmax reached (50000), FE_check starts when i=50200
Algorithm converged at iteration 54200
Warning, fitted Pareto shape parameter =1.0972024623344898>=0.05!
μ=-118.87378524576447,S=42.73061573935452
inexactLineSearch succeeded, setting initial stepsize to 6.744799667278967e-6
Warning, fitted Pareto shape parameter =1.0976474084859107>=0.05!
μ=-118.881293176378,S=49.06944915376517
inexactLineSearch succeeded, setting initial stepsize to 1.4101700569832182e-5
             VI result might be inaccurate!
μ=-71.33369850991906,S=75.4288631126996


## With Step Size found by Adaptive Step Size

In [11]:
max_iterations = 1e5
for str in ["free_energy","none","MonteCarlo"]
opt = ParamStr2(stepsize_update="adaptive",pareto_k_thr=0.7,max_iterations=max_iterations,mcmc_window_len=2000,
    rhat_cutoff=1.5,mcse_cutoff =1e-3,ess_threshold=150.,convergence_algo=str,verbose=true)
#opt= Descent(1e-7)
λ = renderCVI(logp_nc,Int64(max_iterations),opt,λ_init,msg_in)
λ_bc = naturalToBCParams(msg_in.dist,λ)
println("μ=$(λ_bc[1]),S=$(λ_bc[2])")
end

Burninmax reached (50000), FE_check starts when i=50200
Algorithm converged at iteration 54200
Importance ratios are 0, fitted Pareto shape parameter = NaN
μ=-106.88064641894836,S=109.33044274133975
Importance ratios are 0, fitted Pareto shape parameter = NaN
μ=111.9560668408084,S=105.08045854109831
             VI result might be inaccurate!
μ=23.75599507705373,S=38.42141149669742


## With Step Size decay by Robbins - Monro

In [12]:
max_iterations = 1e5
for str in ["free_energy","none","MonteCarlo"]
opt = ParamStr2(eta=0.5,stepsize_update="decay",pareto_k_thr=0.7,max_iterations=max_iterations,mcmc_window_len=2000,
    rhat_cutoff=1.5,mcse_cutoff =1e-3,ess_threshold=150.,convergence_algo=str,verbose=true)
#opt= Descent(1e-7)
λ = renderCVI(logp_nc,Int64(max_iterations),opt,λ_init,msg_in)
λ_bc = naturalToBCParams(msg_in.dist,λ)
println("μ=$(λ_bc[1]),S=$(λ_bc[2])")
end

FE (711762.9770136658) is smaller now,FE_check starts when i=5200
Algorithm converged at iteration 9200
Warning, fitted Pareto shape parameter =0.8709271258414547>=0.7!
μ=-118.88898488920779,S=100.01000000000114
Warning, fitted Pareto shape parameter =1.063950424341222>=0.7!
μ=118.89237239488084,S=100.01000000000114
             VI result might be inaccurate!
μ=-23.77908966607842,S=100.01000000000187


## Check if Opt is updated after MCMC

In [13]:
max_iterations = 1e4
str = "MonteCarlo"
opt = ParamStr2(stepsize_update="adaptive",pareto_k_thr=0.7,max_iterations=max_iterations,mcmc_window_len=2000,
    rhat_cutoff=1.5,mcse_cutoff =1e-3,ess_threshold=150.,convergence_algo=str,verbose=true)
λ = renderCVI(logp_nc,Int64(max_iterations),opt,λ_init,msg_in)
λ_bc = naturalToBCParams(msg_in.dist,λ)
println("μ=$(λ_bc[1]),S=$(λ_bc[2])")
println(opt.h̄,"---",opt.ḡ,"---",opt.tau)
println(λ,λ_bc)

             VI result might be inaccurate!
μ=21.05205772245176,S=111.4264397113747
1.243104021397104e12---[-180651.3289917953, 100.0]---100.0
[2345.755840611151, -55.71321985568735][21.05205772245176, 111.4264397113747]
